<a href="https://colab.research.google.com/github/shankencedric/cs180proj/blob/main/MantisMinds%E2%84%A2%EF%B8%8F_Bucu_Legara_Project_THX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS 180 Introduction to AI | Final Project**

Group Name: **MantisMinds™️** \
Section: *THX (2:30-4:00pm)* \
Members: \
- **BUCU, Clarisse Bianca C.** ---------- *2020-08925*
- **LEGARA, Sean Ken Cedric G.** ---- *2021-08117* \




